In [1]:
#Michael Timbes
#Purpose:
#Image_Classification Based on Logistic classification model. Single hidden layer, does not use convolutional layers
#does not use pooling.

In [2]:
import tensorflow as tf
import numpy as np
#Visualization
import matplotlib.pyplot as plt
#File import
from os import listdir
from os import path as opath
from PIL import Image as PImage

In [3]:
#Path of train images
train_path = ('trainset2')
#Path of test images
test_path = ('testset2')

#Dimension of image
WIDTH = 100
HEIGHT = 100

#Trainig rate alpha
alpha = 0.1

#Number of inputs defined
NUM_IN = WIDTH * HEIGHT

#Number of classifications
y_out_clss = 2
keyA = 'face' #True Class
keyB = 'notface' #False Class

#Size of batch
BatchSize = 2

In [4]:
#Import data
def ImportImages(path, width, height):
    """
    Images must be in RGB space.
    ________________________________________________________________
    Function Outline:
    1. Loads list of images from the path variable (type string).
    2. Iterates through directory loads image into I/O file stream.
    3. Converts file Numpy array.
    4. Reads labels and converts to binary class.
    5. Returns Numpy array objects for images and labels.
    """

    loadedImages = []
    loadedLabels = []
    originalLabels = []
    
    imagesList = listdir(path)
  
    new_size = width, height
    
    for image in imagesList:
        if not(".DS_Store" in path +'/'+ image): #Issue in Mac File System
            img = PImage.open(path +'/'+ image)
            label = opath.splitext(image)[:]
        
            # Pull file name from current image- use it as a label
            originalLabels.append(label[0])
            img.load()

            # Resize step- ensures that all images follow.
            #img.thumbnail(new_size, PImage.ANTIALIAS )
            #img.convert('1')
            #img.resize(new_size)
            loadedImages.append(np.asarray( img, dtype="int32" ))
        
    # Convert to Binary Classification.
    for originalLabel in originalLabels:
        if keyA in originalLabel and not(keyB in originalLabel):
            loadedLabels.append([1, 0])
        else:
            loadedLabels.append([0, 1])
    
    return np.asarray(loadedImages), np.asarray(loadedLabels), originalLabels

In [5]:
def shape_up_(data, width):
    """
    Expects a NUM_IN * NUM_IN sized picture.
    Changes the shape to be (N,NUM_IN**2).
    """
    num_exs = len(data[:,0,0,0])
    new_X = np.zeros((num_exs,width**2 * 3))
    for i in range(0,num_exs):
        new_X[i,:] = data[i,:,:,:].reshape((1,width**2 * 3))
    return new_X

In [6]:
def out_class(Y):
    """
    Matches Class with input vector.
    """
    return np.asarray([keyB if label == 1 else keyA for label in Y])

# Data Preparation
* Import Images
* Reshape Images
* Final Array Should Be:
$$train_{x} = \begin{pmatrix} 
[Picture Array_{MxM}] , & [Num Examples]\\
\end{pmatrix}$$
$$train_{y} = \begin{pmatrix} 
Class_{1} & Class_{2} & \cdots & Class_{n}\\
\end{pmatrix}$$
Where the $Class_{n}$ is either a 1 for the true class or 0 for the false class.

In [7]:
#Training Data Preparation
training_images, training_labels, training_original = ImportImages(train_path, WIDTH, HEIGHT)
print(training_images.shape)
m = len(training_images)
print("Number of examples: ", m)
training_images = shape_up_(training_images,WIDTH)
# Showing the shape of train_X
print(training_images.shape)
print(training_labels)
print(training_original)

(18, 100, 100, 3)
Number of examples:  18
(18, 30000)
[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]
['face_0', 'face_1', 'face_2', 'face_90', 'face_91', 'face_92', 'face_93', 'face_94', 'notface_0', 'notface_1', 'notface_2', 'notface_3', 'notface_4', 'notface_95', 'notface_96', 'notface_97', 'notface_98', 'notface_99']


In [8]:
#Testing Data Preparation 
test_images, test_labels, test_original = ImportImages(test_path, WIDTH, HEIGHT)
test_images = shape_up_(test_images, WIDTH)
print(test_images.shape)
print(test_labels)
print(test_original)

(6, 30000)
[[1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]]
['face_11', 'face_12', 'face_13', 'notface_17', 'notface_18', 'notface_19']


# Build Logistic Model
## Input Layer
Dimension for $X$ is $1xN$. For Tensorflow, x_input_layer as a placeholder must be at least a 1-D vector but can support $MxN$ so a 'None' type is used to be more dynamic. To ensure the matrix multiplication is not an issue be sure that the weight layer and X layer are $Nx1$ and $1xN$. 
\begin{equation}
    \begin{pmatrix}
    x_{0} \\
    x_{1} \\
    \vdots \\
    x_{n}
    \end{pmatrix}
\end{equation}
## Weights
\begin{equation}
    \begin{pmatrix}
    \theta_{0} & \theta_{1} & \cdots & \theta{n} \\
     & & \vdots & \\
     \theta_{0} & \theta_{1} & \cdots & \theta{n} \\
    \end{pmatrix}
\end{equation}

## Output Layer
Below is for multi-class in this application where there are $\theta_{n}$ classes.
\begin{equation}
h_{\theta}(x) = 
\begin{pmatrix}
    \theta_{0} & \theta_{1} & \cdots & \theta{n} \\
     & & \vdots & \\
     \theta_{0} & \theta_{1} & \cdots & \theta{n} \\
    \end{pmatrix}
    \begin{pmatrix}
    x_{0} \\
    x_{1} \\
    \vdots \\
    x_{n}
    \end{pmatrix} =
     \begin{pmatrix}
    y_{0} \\
    y_{1} \\
    \vdots \\
    y_{m}
    \end{pmatrix}
\end{equation}

In [9]:
#The x_input_layer and y_output_layer values are placeholders for the model that accept the flattened image (x) 
#and then the ouput of theclassifications (y). 
#
x_input_layer = tf.placeholder(tf.float32, shape=[None, NUM_IN * 3]) 
y_ = tf.placeholder(tf.float32, shape= [None, y_out_clss])


#The vectors for weights and b- the bias will be defined as variables for training later.

Weights = tf.Variable(tf.zeros([NUM_IN * 3, y_out_clss]))
b = tf.Variable(tf.zeros([y_out_clss]))

In [10]:
#Outline of the model based on the probabilities calculated plus bias values.

y = tf.matmul(x_input_layer, Weights) + b

# Cross Entropy
## Cost Function
\begin{equation}
J(\theta)= -\frac{1}{m}\sum_{i=1}^{m} y_{i}log(h_{\theta}(x_{i}))+
									  (1-y_{i})log(1-h_{\theta}(x_{i}))
\end{equation}
## Minimize Cost-Gradient Descent
\begin{equation}
\theta_{j} = \theta_{j}-\alpha\frac{1}{m}\sum_{i=1}^{m}\left((h_{\theta}(x_{i})-y_{i})X_{ji}\right)
\end{equation}

In [11]:
#This is where the train steps happens. Cross entropy is calculated by running the current model and then running 
#gradient decent. Training step stores results from the gradient descent minimizing cost function (cross_entropy).


cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_, logits = y))

train_step = tf.train.GradientDescentOptimizer(alpha).minimize(cross_entropy)

In [12]:
#Init session and global variables
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
#Design batch variables- ensure dimension
#x_batch = np.ones((1, NUM_IN))
#y_batch = np.ones((1, 2))

In [13]:
#Training Block:
for i in range(1000): #Outside train loop
     for j in range(0,m):
            sess.run(train_step,feed_dict={x_input_layer: training_images, y_: training_labels})
print("DONE.")

DONE.


In [14]:
#Test Block:

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x_input_layer: test_images, y_: test_labels}))
y_out = sess.run(tf.argmax(y,1),feed_dict={x_input_layer: test_images})

# Ouput of Tests
print(y_out)
classes = out_class(y_out)
print("OUTPUT: ", classes)
print("ACTUAL: ", test_original)

1.0
[0 0 0 1 1 1]
OUTPUT:  ['face' 'face' 'face' 'notface' 'notface' 'notface']
ACTUAL:  ['face_11', 'face_12', 'face_13', 'notface_17', 'notface_18', 'notface_19']
